In [17]:
import sys
import pandas as pd
import numpy as np
from time import time
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor, Perceptron
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVC, SVR
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from joblib import Memory
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from ml_metrics import quadratic_weighted_kappa

def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)

#TODO: check which ones are really worth encoding and which can be even dropped (some may contain nonoverlapping values between
#test and training sest)
categorical = {'Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 
               'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 
               'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 
               'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 
               'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 
               'Insurance_History_9', 'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 
               'Medical_History_4', 'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 
               'Medical_History_8', 'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 
               'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 
               'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 
               'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 
               'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 
               'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 
               'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 
#                'Medical_History_38', 
               'Medical_History_39', 'Medical_History_40', 'Medical_History_41','Medical_History_1', 
               'Medical_History_15', 'Medical_History_24', 'Medical_History_32'}

In [18]:
import logging
# create logger
logger = logging.getLogger("logging_tryout2")
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)

# create formatter
# formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s")
formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s",
                              "%Y-%m-%d %H:%M:%S")
# add formatter to ch
ch.setFormatter(formatter)
# logger.addHandler(ch)
def info(msg):
    logger.info(msg.replace("\n", "  "))

In [19]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
total = pd.concat([train, test])
median = total.median()
train.fillna(median, inplace=True)
test = test.fillna(median, inplace=True)
encoder = LabelEncoder()
for f in categorical:
    encoder.fit(total[f])
    train[f] = encoder.transform(train[f])
    test[f] = encoder.transform(test[f])

In [20]:
feature_cols = test.columns[1:]
categorical_inds = [i for i, col in enumerate(feature_cols) if col in categorical]
oh_encoder = OneHotEncoder(categorical_features=categorical_inds)

X = np.array(train[test.columns[1:]])
y = np.array(train.Response)
X_actual_test = np.array(test[feature_cols])

oh_encoder.fit(X)
X = oh_encoder.transform(X).todense()
X_actual_test = oh_encoder.transform(X_actual_test).todense()

train_test_folds = list(StratifiedKFold(y, n_folds=6, random_state=0))

In [21]:
train_cache = Memory(cachedir="cache/_train", verbose=0)
test_cache = Memory(cachedir="cache/test", verbose=0)

@train_cache.cache
def train_predictions(model):
    ind2pred = {}
    for i, (train, test) in enumerate(train_test_folds):
        info(("fitting fold   "+str(i+1)+ str(model)[:100]))
        model.fit(X[train], y[train])
        info(("fold fitted    "+str(i+1)+  str(model)[:100]))
        preds = model.predict(X[test])
        for i, p in zip(test, preds):
            ind2pred[i] = p
    
    return np.array([ind2pred[i] for i in range(len(y))])

@test_cache.cache
def test_predictions(model):
    model.fit(X, y)
    return model.predict(X_actual_test)


stacker_train_cache = Memory(cachedir="cache/stacker_train", verbose=0)
stacker_test_cache = Memory(cachedir="cache/stacker_test", verbose=0)

@stacker_train_cache.cache
def stacker_train_predictions(stacker, base_clfs):
    info("start stacker --------------------------")
    n = len(y)
    stacked_X = np.hstack([X] + [train_predictions(clf).reshape(n, 1) for clf in base_clfs])
    info("base regressors done")
    ind2pred = {}
    for i, (train, test) in enumerate(train_test_folds):
        info("fitting stacker fold %s   %s" % (i, str(stacker)))

        stacker.fit(stacked_X[train], y[train])
        info("stacker fitted fold %s    %s " % (i, str(stacker)))
        preds = stacker.predict(stacked_X[test])
        for i, p in zip(test, preds):
            ind2pred[i] = p
    info("stacker done =========================")
    return np.array([ind2pred[i] for i in range(len(y))])

@stacker_test_cache.cache
def stacker_test_predictions(stacker, base_clfs):
    n = len(y)
    stacked_X = np.hstack([X] + [train_predictions(clf).reshape(n, 1) for clf in base_clfs])
    stacker.fit(stacked_X, y)
    n = X_actual_test.shape[0]
    stacked_test_X = np.hstack([X_actual_test] + [test_predictions(clf).reshape(n, 1) for clf in base_clfs])
    return stacker.predict(stacked_test_X)

In [22]:
def benchmark(model):
    pred = train_predictions(model)
    return eval_wrapper(pred, y)

def make_predictions(model):
    model.fit(X, y)
    return model.predict(X_actual_test)

def benchmark_stacker(model, base_clfs):
    pred = stacker_train_predictions(model, base_clfs)
    return eval_wrapper(pred, y)


In [23]:
xgbr = XGBRegressor(objective="reg:linear", min_child_weight=80, subsample=0.85, colsample_bytree=0.30, silent=1, max_depth=9)
xgbc = XGBClassifier(objective="reg:linear", min_child_weight=80, subsample=0.85, colsample_bytree=0.30, silent=1, max_depth=9)
rfr = RandomForestRegressor(n_estimators=400)
etr = ExtraTreesRegressor(n_estimators=400)
sgdr = SGDRegressor()

In [8]:
%%time
benchmark_stacker(xgbc, [xgbr])

2016-01-16 15:49:41;INFO;start stacker
INFO:logging_tryout2:start stacker
2016-01-16 15:49:41;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:51:52;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:51:52;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:54:03;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, co

6701.41893506


In [ ]:
benchmark_stacker(xgbc, [xgbr])

In [ ]:
%%time
benchmark_stacker(xgbc, [LinearRegression()])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="poly")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="rbf")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="sigmoid")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [Perceptron()])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [rfrg])

In [ ]:
%%time
benchmark_stacker(xgbc, [rfre])

In [ ]:
%%time
benchmark_stacker(xgbc, [etrg])

In [ ]:
%%time
benchmark_stacker(xgbc, [etre])

In [ ]:
%%time
benchmark_stacker(xgbc, [sgdr])

In [9]:
benchmark_stacker(xgbc, [xgbr, rfr,  etr, sgdr, LinearRegression(), Perceptron(),
                                                 SVR(kernel="linear"), SVR(kernel="poly"), SVR(kernel="sigmoid"),
                                                 SVR(kernel="rbf")])

NameError: name 'rfre' is not defined

In [ ]:
super_predictions = stacker_test_predictions(xgbc[xgbr, rfre, rfrg, etre, etrg, sgdr, LinearRegression(), Perceptron(),
                                                 SVR(kernel="linear"), SVR(kernel="poly"), SVR(kernel="sigmoid"),
                                                 SVR(kernel="rbf")])

In [ ]:
def make_sub(stacker, base_clfs, filename):
    preds = stacker_test_predictions(stacker, base_clfs)
    
    df = pd.DataFrame()
    df['Id'] = test.Id
    df['Response'] = preds

    df.to_csv(filename, index=False)

In [1]:
1 +1

2

In [14]:
%%time
n=10000
etr.fit(X[:n], y[:n])

CPU times: user 13min 36s, sys: 68.8 ms, total: 13min 36s
Wall time: 13min 36s


In [13]:
np.random.rand(100)

array([ 0.9565489 ,  0.06646688,  0.13886124,  0.40147266,  0.2553984 ,
        0.623276  ,  0.84264077,  0.49614786,  0.66286836,  0.18527387,
        0.84390153,  0.52411654,  0.26703591,  0.95946522,  0.09500017,
        0.08819645,  0.06444954,  0.73305136,  0.87945592,  0.42660323,
        0.35605152,  0.02288685,  0.10615171,  0.94288078,  0.98991951,
        0.38985553,  0.0814427 ,  0.87975251,  0.21464408,  0.1758485 ,
        0.64828491,  0.92377805,  0.3330988 ,  0.74079717,  0.25670308,
        0.8055024 ,  0.76828103,  0.68848119,  0.75671415,  0.89444866,
        0.39103324,  0.92278116,  0.52587316,  0.46271616,  0.76237296,
        0.03542513,  0.10724893,  0.4890357 ,  0.24310663,  0.26450881,
        0.6273336 ,  0.75116666,  0.30109729,  0.98394813,  0.90146232,
        0.30760704,  0.85679848,  0.66051435,  0.32339268,  0.86702217,
        0.17667357,  0.84905053,  0.08698123,  0.30957962,  0.34737646,
        0.4304363 ,  0.3021332 ,  0.32132048,  0.89473947,  0.37